In [22]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from conversion import CSVToTensor

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_size = 9
        self.output_size = 9
        self.hidden_size = 2*27

        self.fc1 = nn.Linear(self.input_size, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.crossloss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.train_data = None
        self.val_data = None
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

    def load_data(self, file_path):
        dataloader = CSVToTensor(file_path)
        dataloader.create_all_tensor()
        dataset = dataloader.create_a_dataset()

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_data, self.val_data = random_split(dataset, [train_size, val_size])
        self.train_data = DataLoader(self.train_data, batch_size=16, shuffle=True)
        self.val_data = DataLoader(self.val_data, batch_size=16, shuffle=True)

    def train_model(self, epochs):
        self.to(self.device)
        self.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for src, trg in self.train_data:
                src = src.to(self.device)
                trg = trg.to(self.device)
                self.optimizer.zero_grad()
                output = self.forward(src)
                loss = self.crossloss(output, trg.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(self.train_data)
            print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}")

    

In [23]:
if __name__ == '__main__':
    model = Model()
    # choose the dataset file path
    model.load_data('./Datasets/tic_tac_toe_500_games.csv')
    # choose the number of epochs
    model.train_model(100)

Epoch: 0, Loss: 2.1892
Epoch: 1, Loss: 2.1175
Epoch: 2, Loss: 2.0562
Epoch: 3, Loss: 2.0076
Epoch: 4, Loss: 1.9573
Epoch: 5, Loss: 1.8986
Epoch: 6, Loss: 1.8920
Epoch: 7, Loss: 1.8591
Epoch: 8, Loss: 1.8455
Epoch: 9, Loss: 1.8229
Epoch: 10, Loss: 1.8310
Epoch: 11, Loss: 1.8136
Epoch: 12, Loss: 1.8074
Epoch: 13, Loss: 1.7787
Epoch: 14, Loss: 1.7838
Epoch: 15, Loss: 1.7880
Epoch: 16, Loss: 1.7516
Epoch: 17, Loss: 1.7759
Epoch: 18, Loss: 1.7697
Epoch: 19, Loss: 1.7578
Epoch: 20, Loss: 1.7605
Epoch: 21, Loss: 1.7469
Epoch: 22, Loss: 1.7324
Epoch: 23, Loss: 1.7509
Epoch: 24, Loss: 1.7429
Epoch: 25, Loss: 1.7396
Epoch: 26, Loss: 1.7277
Epoch: 27, Loss: 1.7438
Epoch: 28, Loss: 1.7388
Epoch: 29, Loss: 1.7290
Epoch: 30, Loss: 1.7398
Epoch: 31, Loss: 1.7419
Epoch: 32, Loss: 1.7242
Epoch: 33, Loss: 1.7247
Epoch: 34, Loss: 1.7238
Epoch: 35, Loss: 1.7406
Epoch: 36, Loss: 1.7253
Epoch: 37, Loss: 1.7241
Epoch: 38, Loss: 1.7263
Epoch: 39, Loss: 1.7214
Epoch: 40, Loss: 1.7167
Epoch: 41, Loss: 1.7080
Ep